In [1]:

import pandas as pd
import numpy as np
import warnings

import os
import holidays
from datetime import datetime
warnings.filterwarnings('ignore')

Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: Não foi possível encontrar o procedimento especificado.


In [2]:
#parametros de conexao
PASSWORD = os.environ.get("senha_sf")
WAREHOUSE = "EV"
ACCOUNT = "ot57980.us-east-2.aws"
DATABASE = "rede_vistorias"
SCHEMA = "dbt_rvdata_marts"
USER=os.environ.get("usuario_sf")

connection_parameters = {
   "account": ACCOUNT,
   "user": USER,
   "password": PASSWORD,
   "role": "ACCOUNTADMIN",
   "warehouse": WAREHOUSE,
   "database": DATABASE,
   "schema": SCHEMA}

In [3]:
#creating a snowflake session
prod = Session.builder.configs(connection_parameters).create()
prod.add_packages('snowflake-snowpark-python','pandas','numpy','scikit-learn','holidays')

The version of package 'snowflake-snowpark-python' in the local environment is 1.6.1, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'numpy' in the local environment is 1.24.4, which does not fit the criteria for the requirement 'numpy'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'holidays' in the local environment is 0.22, which does not fit the criteria for the requirement 'holidays'. Your UDF might not work when the package version is different between the server and your local environment.


In [4]:
from snowflake.snowpark.types import IntegerType, StringType, StructType, StructField,TimestampType

In [5]:
def snowflake_function(session: snowflake.snowpark.Session) -> str:

    #puxar a data atual para delimitar o filtro de feriados
    session.add_packages('snowflake-snowpark-python','pandas','numpy','holidays')

    query_current_date = '''
    select current_date() as current_date
    '''
    df = session.sql(query_current_date).to_pandas()

    lista_feriados = []
    feriados= holidays.Brazil()
        
    #colocando os feriados em uma lista    
    for feriado in feriados['2018-01-01': df['CURRENT_DATE'][0].strftime("%m/%d/%Y")] :
        lista_feriados.append(feriado)

    #query dados das vistorias
    query = '''
    select ID,CREATED_AT,FIRST_SENT_AT,
    FRANCHISEE_ID,CLIENT_ID, TYPE, to_date(FIRST_SENT_AT) as date_only,
    credits/100 as credits
    from orders 
    where CANCELED_AT is null and FIRST_SENT_AT  > '2022-06-01' and (data['accompaniedInspection']['scheduleDate'] is null
    OR data['details']['allowed_at_date'] is null)
    ;
    '''
    df = session.sql(query).to_pandas()


    def tempo_de_entrega(df):

        horas_entrega = []
        lista_atraso = []
        for index,row in df.iterrows():
            start = row['CREATED_AT'].date()
            end = row['FIRST_SENT_AT'].date()
            segundos = row['FIRST_SENT_AT'] -  row['CREATED_AT']
            horas = segundos.total_seconds() / 60/60

            # include holidays in a list
            days = np.busday_count(start, end, holidays=lista_feriados)
            tempo = round((days*24)+horas)
            horas_entrega.append(tempo)

            if row['CREATED_AT'].hour > 12:
                if tempo > 72:
                    lista_atraso.append('Atrasado')
                else:
                    lista_atraso.append('Em tempo')
            else:
                if tempo > 48:
                    lista_atraso.append('Atrasado')
                else:
                    lista_atraso.append('Em tempo')   

        df['atraso']=lista_atraso
        df['horas_entrega']=horas_entrega

        return df
    
    df = tempo_de_entrega(df)

    #--------------------------------------------------------------------------------------------------------------------
    #salvando a tabela
    df['CREATED_AT'] = df['CREATED_AT'].dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    df['FIRST_SENT_AT'] = df['FIRST_SENT_AT'].dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    lista_types = [StructField("ID", StringType()),StructField("CREATED_AT", TimestampType()),StructField("FIRST_SENT_AT", TimestampType()),
         StructField("FRANCHISEE_ID", StringType()), StructField("CLIENT_ID", StringType()),StructField("TYPE", StringType()),
         StructField("DATE_ONLY", TimestampType()),StructField("CREDITS", TimestampType())]
    tschema = StructType(lista_types)
    df_snow = session.create_dataframe(data = df,schema = tschema)
    df_snow.write.mode("overwrite").save_as_table("tempo_entrega", mode="overwrite", table_type="")

    return "SUCCESS"


my_copy_sp = prod.sproc.register(snowflake_function, name="rede_vistorias.dbt_rvdata_marts.tempo_entrega_v1_old", replace=True,
is_permanent=True, stage_location="@REDE_VISTORIAS.DBT_RVDATA.s3_churn")

ProgrammingError: 091003 (22000): Failure using stage area. Cause: [S3_CHURN GET and PUT commands are not supported with external stage]

VERSAO 2.0: CONTEMPLA A DATA DE LIBERAÇÃO

In [6]:
def snowflake_function(session: snowflake.snowpark.Session) -> str:

    #puxar a data atual para delimitar o filtro de feriados
    session.add_packages('snowflake-snowpark-python','pandas','numpy','holidays')

    query_current_date = '''
    select current_date() as current_date
    '''
    df = session.sql(query_current_date).to_pandas()

    lista_feriados = []
    feriados= holidays.Brazil()
        
    #colocando os feriados em uma lista    
    for feriado in feriados['2018-01-01': df['CURRENT_DATE'][0].strftime("%m/%d/%Y")] :
        lista_feriados.append(feriado)

    #query dados das vistorias
    query = '''
    select ID,CREATED_AT,FIRST_SENT_AT,
    FRANCHISEE_ID,CLIENT_ID, TYPE, to_date(FIRST_SENT_AT) as date_only,
    credits/100 as credits,
    regexp_replace(replace(data['accompaniedInspection']['scheduleDate'],' ','')::varchar(), '[A-Z]', '') as data_agendamento,
    regexp_replace(replace(data['details']['allowed_at_date'],' ','')::varchar(), '[A-Z]', '') as data_liberacao
    from orders 
    where CANCELED_AT is null and FIRST_SENT_AT  > '2022-06-01' order by created_at desc
    ;
    '''
    df = session.sql(query).to_pandas()

    def tratar_data(df,coluna_base):
        lista_data = []

        for x in df[coluna_base]:
            try:
                
                lista_data.append(datetime.strptime(x, '%d/%m/%Y%H:%M'))
            except:    
                try:
                    lista_data.append(datetime.strptime(x, '%d/%m/%y'))    
                except:
                    try:        
                        lista_data.append(datetime.strptime(x, '%d/%m/%Y'))     
                    except:
                        lista_data.append(None)
                        

        return lista_data

    #teste validade anos
    lista_primeiro_ajuste = []
    for x in df['DATA_LIBERACAO']:
        try:
            if x is not None and (int(x.split('/')[-1].split(':')[0][0:4]) > 2024 or int(x.split('/')[-1].split(':')[0][0:4]) < 2020) and len(x.split('/')[-1].split(':')[0]) > 2:
                lista_primeiro_ajuste.append(None)
            else:
                lista_primeiro_ajuste.append(x)
        except:
            pass
            lista_primeiro_ajuste.append(x)

    df['DATA_LIBERACAO'] = lista_primeiro_ajuste

    lista_primeiro_ajuste = []
    for x in df['DATA_AGENDAMENTO']:
        try:
            if x is not None and (int(x.split('/')[-1].split(':')[0][0:4]) > 2024 or int(x.split('/')[-1].split(':')[0][0:4]) < 2020) and len(x.split('/')[-1].split(':')[0]) > 2:
                lista_primeiro_ajuste.append(None)
            else:
                lista_primeiro_ajuste.append(x)
        except:
            pass
            lista_primeiro_ajuste.append(x)

    df['DATA_AGENDAMENTO'] = lista_primeiro_ajuste
    
    df['DATA_AGENDAMENTO_TRATADA'] = tratar_data(df,'DATA_AGENDAMENTO')
    df['DATA_AGENDAMENTO_TRATADA'].fillna(df['CREATED_AT'],inplace = True)
    df['DATA_LIBERACAO_TRATADA'] = tratar_data(df,'DATA_LIBERACAO')
    df['DATA_LIBERACAO_TRATADA'].fillna(df['CREATED_AT'],inplace = True)

    def tempo_de_entrega(df):

        horas_entrega = []
        lista_atraso = []
        lista_used_date = []
        for index,row in df.iterrows():
            used_date = max(row['CREATED_AT'],row['DATA_LIBERACAO_TRATADA'],row['DATA_AGENDAMENTO_TRATADA'])
            if used_date > row['FIRST_SENT_AT']:
                used_date = row['CREATED_AT']
            start = used_date.date()
            end = row['FIRST_SENT_AT'].date()
            segundos = row['FIRST_SENT_AT'] -  used_date
            horas = segundos.total_seconds() / 60/60
            lista_used_date.append(used_date)

            # include holidays in a list
            days = np.busday_count(start, end, holidays=lista_feriados)
            total_dias = (end - start).days
            tempo = round(horas-((total_dias - days)*24))
            horas_entrega.append(tempo)

            if row['CREATED_AT'].hour > 12:
                if tempo > 72:
                    lista_atraso.append('Atrasado')
                else:
                    lista_atraso.append('Em tempo')
            else:
                if tempo > 48:
                    lista_atraso.append('Atrasado')
                else:
                    lista_atraso.append('Em tempo')   

        df['atraso']=lista_atraso
        df['horas_entrega']=horas_entrega
        df['used_date'] = lista_used_date

        return df
    
    df = tempo_de_entrega(df)

    #--------------------------------------------------------------------------------------------------------------------
    #salvando a tabela
    df['CREATED_AT'] = df['CREATED_AT'].dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    df['FIRST_SENT_AT'] = df['FIRST_SENT_AT'].dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    df['used_date'] = df['used_date'].dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    df['DATA_LIBERACAO_TRATADA']=df['DATA_LIBERACAO_TRATADA'].dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    df['DATA_AGENDAMENTO_TRATADA']=df['DATA_AGENDAMENTO_TRATADA'].dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    lista_types = [StructField("ID", StringType()),StructField("CREATED_AT", TimestampType()),StructField("FIRST_SENT_AT", TimestampType()),
         StructField("FRANCHISEE_ID", StringType()), StructField("CLIENT_ID", StringType()),StructField("TYPE", StringType()),
         StructField("DATE_ONLY", TimestampType()),StructField("CREDITS", StringType),
         StructField("data_agendamento", StringType()),StructField("data_liberacao", StringType()),StructField("used_date", StringType())]
    tschema = StructType(lista_types)
    df_snow = session.create_dataframe(data = df,schema = tschema)
    df_snow.write.mode("overwrite").save_as_table("tempo_entrega_v2", mode="overwrite", table_type="")

    return "SUCCESS"


my_copy_sp = prod.sproc.register(snowflake_function, name="rede_vistorias.dbt_rvdata_marts.tempo_entrega", replace=True,
is_permanent=True, stage_location="@REDE_VISTORIAS.DBT_RVDATA.s3_churn")

ProgrammingError: 091003 (22000): Failure using stage area. Cause: [S3_CHURN GET and PUT commands are not supported with external stage]